In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../../../data_analysis/')
from simulate_door_slam import *
from trajectory import *
from utils import *
from local_plot import *

def read_frame_timestamps(g2o_path, nodes):
    frame_id_to_stamp = {}
    for i in nodes:
        with open(f"{g2o_path}/frame_timestamp_{i}.txt", "r") as f:
            for line in f:
                frame_id, stamp = line.split()
                frame_id_to_stamp[int(frame_id)] = float(stamp)
    return frame_id_to_stamp

def process_output(g2o_path, nodes):
    poses_pgo, t0 = read_paths(g2o_path, nodes, prefix="pgo_")
    paths_gt, _ = read_paths(g2o_path + "/../", nodes, prefix="groundtruth_", t0=t0)
    align_paths(poses_pgo, paths_gt, align_by_first=True, align_with_minize=True)
    display(plot_fused_err(nodes, poses_pgo, paths_gt, show=False))
    # plot_fused(nodes, poses_pgo, paths_gt)
    dte = 550
    display(relative_pose_err(nodes, poses_pgo, paths_gt, dte=dte))


# Note
We need to start ROS Core Manually for this file

In [ ]:
def evaluate_pgo(g2o_path, g2o_input, max_steps=100, eta_k=1.45, rho_frame_T=0.39, 
            rho_frame_theta=1.556, simulate_delay_ms=0.0, verify_accuaracy=False, agent_num=5, dgs_max_steps=50, 
            max_solving_time=10.0, rho_rot_mat=0.09):
    pg = PoseGraph(g2o_input)
    ignore_infor = False
    print(f"Initial cost: {pg.evaluate_cost():.1f} edges {len(pg.edges)}")
    if verify_accuaracy:
        frame_id_to_stamp = read_frame_timestamps(g2o_path, range(agent_num))
    output_path=g2o_path + "/d2pgo-rot-inited/"
    pgo_optimized, ret = call_d2pgo_opti(g2o_folder=g2o_input, output_folder=output_path, enable_rot_init=True, 
        max_steps=max_steps, agent_num=agent_num, ignore_infor=ignore_infor, eta_k=eta_k, rho_frame_theta=rho_frame_theta, 
        rho_frame_T=rho_frame_T, rho_rot_mat=rho_rot_mat, simulate_delay_ms=simulate_delay_ms, max_solving_time=max_solving_time)
    if verify_accuaracy:
        pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
        process_output(output_path, [0, 1, 2, 3, 4])
    print("Avg Traj len: ", pgo_optimized.evaluate_trajectory_length())
    print(f"D2PGO RotInit cost:         {pgo_optimized.evaluate_cost():.5f} solve_time {ret['max_solve_time']:.1f} ms iters {ret['mean_iters']}")

    output_path=g2o_path + "/d2pgo-no-rot-init/"
    pgo_optimized, ret  = call_d2pgo_opti(g2o_folder=g2o_input, output_folder=output_path, enable_rot_init=False, 
        max_steps=max_steps, agent_num=agent_num, ignore_infor=ignore_infor, eta_k=eta_k, rho_frame_theta=rho_frame_theta, 
        rho_frame_T=rho_frame_T, rho_rot_mat=rho_rot_mat, simulate_delay_ms=simulate_delay_ms, max_solving_time=max_solving_time)
    print(f"D2PGO without RotInit cost: {pgo_optimized.evaluate_cost():.5f} solve_time {ret['max_solve_time']:.1f} ms iters {ret['mean_iters']}")
    if verify_accuaracy:
        pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
        process_output(output_path, [0, 1, 2, 3, 4])
    # output_path = g2o_path + "/DGS/"
    # iterations, min_time, max_time, initial, final, util_rate, total_optim = call_DGS_solver(g2o_input, agent_num=agent_num, 
    #     rthresh=1e-4, pthresh=1e-4, maxIter=dgs_max_steps, between_noise="true")
    # pgo_optimized = copy.copy(pg)
    # pgo_optimized.read_g2o_single(g2o_input+"/fullGraph_optimized.g2o", update_only=True)
    # total_time = max_time + iterations*simulate_delay_ms #Assume communication time is 100ms
    # print(f"DGS cost: {pgo_optimized.evaluate_cost():.1f} max_time: {max_time} iter {iterations} total_time {total_time} DGS cost initial {initial} final: {final} ")
    # if verify_accuaracy:
    #     pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
    #     process_output(output_path, [0, 1, 2, 3, 4])


In [ ]:
g2o_path="/home/xuhao/data/d2slam/pgo/tum_corr_5/"
g2o_input = g2o_path + "/input/"
max_steps = 1000
eta_k=1.5101010101010102
rho_frame_T=0.4526572657265727
rho_frame_theta=2.868058805880588
rho_rot_mat =0.0918787878787879
simulate_delay_ms=50.0
max_solving_time =20.0

evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat, dgs_max_steps=40)

In [ ]:
g2o_path="/home/xuhao/data/d2slam/pgo/parking-garage/"
g2o_input = g2o_path + "/input/"
max_steps = 500
evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat)

In [ ]:
g2o_path="/home/xuhao/data/d2slam/pgo/cubicle/"
g2o_input = g2o_path + "/input/"
evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat)

In [ ]:
g2o_path="/home/xuhao/data/d2slam/pgo/OmniLong5Yaw/"
g2o_input = g2o_path + "/input/"
max_solving_time = 20
evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat)

In [ ]:
g2o_path="/home/xuhao/data/d2slam/pgo/ri_realsense_walkaround_2022_10/"
g2o_input = g2o_path + "/input/"
max_solving_time = 20
max_steps = 1000
eta_k=1.5101010101010102
rho_frame_T=0.4526572657265727
rho_frame_theta=2.868058805880588
rho_rot_mat =0.0918787878787879
simulate_delay_ms=50.0
max_solving_time=3.0

evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, simulate_delay_ms, 
                    max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat, agent_num=3)